In [ ]:
import numpy as np
import cv2

In [ ]:
depth_path = r'C:\Users\Furihata\Desktop\Adam\Grasp Planning\Simulation\Real Depth Image\3\1.png'

In [ ]:
image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
image = 255 - image
filt = cv2.GaussianBlur(image, (11,11), 0)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image, cmap = 'gray')
plt.show()

In [ ]:
edge_im = cv2.Canny(image, 10, 50)

In [ ]:
image.shape

In [ ]:
plt.imshow(edge_im,  cmap='Greys')
plt.show()

In [ ]:
image.shape

In [ ]:
image_cropped = image[124:1024, 380:1280]
image_cropped = cv2.resize(image_cropped, (224, 224))

In [ ]:
plt.imshow(image_cropped, cmap = 'gray')
plt.show()

In [ ]:
import Img_antipodal_sampler as spl

In [ ]:
image_filt = cv2.GaussianBlur(image_cropped, (5,5), 0)

In [ ]:
edge_im = cv2.Canny(image_filt, 5, 60)

In [ ]:
normal_px, edge_px, = spl.normals(image_cropped, edge_im)

In [ ]:
grasps, grasp_normals, centers = spl.antipodal_sampler(edge_px, normal_px)

In [ ]:
save_arr = r'C:\Users\Furihata\Desktop\Adam\Grasp Planning\Simulation\Image\realgrasps.png'
depth_loc = r'C:\Users\Furihata\Desktop\Adam\Grasp Planning\Simulation\Image\realdepth.png'

In [ ]:
plt.imsave(depth_loc, image_cropped, cmap = 'gray')

In [ ]:
spl.draw_grasps(grasps, depth_loc, save_arr)

In [1]:
import cv2
import matplotlib.pyplot as plt

In [2]:
def collision_est(depth, seg, ID, grasps, width):
    collision_scores = []
    #epth_boundings = []
   #seg_boundings = []
   #depth_fulls = []
   #seg_fulls = [] 
    for i in range(len(grasps)):
        
        seg = seg.astype('float32')
        depth = depth.astype('float32')
        
        grasp_vector =  grasps[i,1] - grasps[i,0]
        grasp_center = ((grasps[i,0] + grasps[i,1]) / 2).astype(int)
        angle = np.arctan2(grasp_vector[1], grasp_vector[0]) * 180 / np.pi
        
        M = cv2.getRotationMatrix2D((grasp_center[0], grasp_center[1]), angle, 1.0)
        
        depth_im_val = cv2.warpAffine(depth, M, (depth.shape))
        seg_im_val = cv2.warpAffine(seg, M, (seg.shape))
        
        left = int(grasp_center[0] - (width/2))
        right = int(grasp_center[0] + (width/2))    
        top = grasp_center[1] - 30
        bottom = grasp_center[1] + 30

        #seg_full = seg_im_val[top:bottom, left:right]
        #depth_full = depth_im_val[top:bottom, left:right]

        part_ID = ID[i]
        seg_bounding = seg_im_val[grasp_center[1]-7:grasp_center[1]+7, left:right]
        depth_bounding = depth_im_val[grasp_center[1]-7:grasp_center[1]+7, left:right]
        part_indices = np.where(seg_bounding == part_ID) #in the form of [[row1, row2, .....], [column1, column2,.....]]
        nonpart_indices = np.where(seg_bounding != part_ID) #in the form of [[row1, row2, .....], [column1, column2,.....]]
        seg_bounding[part_indices[0], part_indices[1]] = 1
        seg_bounding[nonpart_indices[0], nonpart_indices[1]] = 0

        part_depth_val = depth_bounding[part_indices[0], part_indices[1]]
       
        treshold = np.mean(part_depth_val)
        
        seg_copy = np.copy(seg_bounding)
        
        cs_indices = np.where((seg_bounding == 0) &
                              (depth_bounding <= treshold))
        seg_copy[cs_indices[0], cs_indices[1]] = 1

        if(len(cs_indices[0]) > 50):
            cl_score = 1
        else:
            cl_score = 0
        
        plt.imshow(depth_bounding, cmap = 'gray_r')
        plt.show()
        plt.imshow(seg_bounding,cmap = 'gray')
        plt.show()
        plt.imshow(seg_copy, cmap = 'gray')
        plt.show()
        print(cl_score)
            
        collision_scores.append(cl_score)
        
            
    return np.asarray(collision_scores) #, depth_boundings, depth_fulls, seg_boundings, seg_fulls

In [3]:
import numpy as np
depthImg = np.load('depth.npy')
segImg = np.load('seg.npy')
grasps = np.load('grasps.npy')
ID = np.load('ID.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'depth.npy'

In [ ]:
collision_scores = collision_est(depthImg, segImg, ID, grasps, 24)

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(grasps)):
    plt.imshow(seg_fulls[i])
    plt.show()
    plt.imshow(depth_fulls[i], cmap = 'gray_r')
    plt.show()
    plt.imshow(seg_boundings[i])
    plt.show()
    plt.imshow(depth_boundings[i], cmap = 'gray_r')
    plt.show()
    print(collision_scores[i], '\n')